# Extracting ART Specific Detector Metadata

In [5]:
import json
import importlib
import inspect
import uuid

def write_detector_info(modules_to_inspect):
    # Initialize a list to store the detector info
    detector_info = []

    # Initialize a set to store the names of the saved detectors
    saved_detectors = set()

    # Iterate over each module
    for module_name in modules_to_inspect:
        # Import the module
        module = importlib.import_module(module_name)

        # Get all classes in the module
        classes = inspect.getmembers(module, inspect.isclass)

        # Iterate over each class
        for class_name, class_ in classes:
            # Get all subclasses of the class
            subclasses = class_.__subclasses__()

            # Iterate over each subclass
            for subclass in subclasses:
                # Check if the detector has already been saved
                if subclass.__name__ in saved_detectors:
                    continue

                # Add the detector to the set of saved detectors
                saved_detectors.add(subclass.__name__)

                # Try to create an instance of the subclass
                try:
                    instance = subclass()
                except TypeError:
                    instance = None

                # Get the subclass info
                info = {
                    "Detector": subclass.__name__,
                    "detector_reference": subclass.__module__ + "." + subclass.__name__,
                    "detector_logic_description": inspect.getdoc(subclass),
                    "substrings": instance.substrings if instance and hasattr(instance, 'substrings') else "N/A",
                    "tags": "",
                    "uri": "",
                    "version": "1.0",
                    "language": None,
                    "id": str(uuid.uuid4())  # Generate a unique UUID
                }
                # Add the subclass info to the detector info list
                detector_info.append(info)

    # Write the detector info to a JSONL file
    with open('art_detectors.jsonl', 'w') as f:
        for item in detector_info:
            f.write(json.dumps(item) + '\n')

In [6]:
# List of modules to inspect
modules_to_inspect = ['autoredteam.detectors.adultdata', 'autoredteam.detectors.advstereo', 
                      'autoredteam.detectors.hallucination', 
                      'autoredteam.detectors.paraphrase', 'autoredteam.detectors.privateinfo']

write_detector_info(modules_to_inspect)